In [12]:
# check the percentages of random same-bar chords in generated without constraints
import pandas as pd
import numpy as np

In [2]:
bart_cs_df = pd.read_csv('tokenized/no_constraints/bart_beam_7_temp_0x0/ChordSymbolTokenizer.csv')
bart_pc_df = pd.read_csv('tokenized/no_constraints/bart_beam_7_temp_0x0/PitchClassTokenizer.csv')
gpt_cs_df = pd.read_csv('tokenized/no_constraints/gpt_beam_7_temp_0x0/ChordSymbolTokenizer.csv')
gpt_pc_df = pd.read_csv('tokenized/no_constraints/gpt_beam_7_temp_0x0/PitchClassTokenizer.csv')

In [13]:
def is_sublist_contiguous(q, d):
    q_len = len(q)
    for i in range(len(d) - q_len + 1):
        if d[i:i + q_len] == q:
            return True
    return False

In [5]:
real = bart_cs_df['real'].iloc[0]
gen = bart_cs_df['generated'].iloc[0]

In [ ]:
# def get_cs_same_bar_percentages(real, gen):
real_tokens = real.split()
gen_tokens = gen.split()

In [ ]:
# get number of bars
num_bars = min(real_tokens.count('<bar>') , gen_tokens.count('<bar>') )
# assume no chord token as constraint so far
chord_token = None
# Find indices of all bars
real_indices = [i for i, val in enumerate(real_tokens) if val == '<bar>']
gen_indices = [i for i, val in enumerate(gen_tokens) if val == '<bar>']
bar_index = -1
tries = 10
chord_token = None
while chord_token == None and tries > 0:
    tries -= 1
    # get a random bar
    rand_bar_num = np.random.randint( num_bars )
    # REAL: Get the index of the rand_bar_num occurrence (zero-based index)
    if len(real_indices) > rand_bar_num+1:
        real_bar_index = real_indices[rand_bar_num]
        real_next_bar_index = real_indices[rand_bar_num+1]
    else:
        # check if there are any bars at all
        if len(real_indices) == 0:
            print('This piece has no bars.')
            break
        # the last bar
        real_bar_index = real_indices[-1]
        real_next_bar_index = len(real_tokens)
    # get all tokens between rand_bar and its next
    real_bar_tokens = real_tokens[real_bar_index:real_next_bar_index]
    # check if bar has a chord
    i = 0
    while i < len(real_bar_tokens):
        if 'position_' in real_bar_tokens[i]:
            # keep position token
            position_token = real_bar_tokens[i]
            # check if chord starts bar
            chord_starts_bar = real_bar_tokens[i] == 'position_0x00'
            if i+1 < len(real_bar_tokens):
                chord_token = real_bar_tokens[i+1]
            break
        i += 1
    # end for
# end while
# GEN: Get the index of the rand_bar_num occurrence (zero-based index)
if len(gen_indices) > rand_bar_num+1:
    gen_bar_index = gen_indices[rand_bar_num]
    gen_next_bar_index = gen_indices[rand_bar_num+1]
else:
    # check if there are any bars at all
    if len(gen_indices) == 0:
        print('This piece has no bars.')
    # the last bar
    gen_bar_index = gen_indices[-1]
    gen_next_bar_index = len(gen_tokens)
# get all tokens between rand_bar and its next
gen_bar_tokens = gen_tokens[gen_bar_index:gen_next_bar_index]